# 일원 분산 분석

주어진 데이터는 4가지 다른 교육 방법을 적용한 대학생들의 학점 결과이다. 이 실험에서는 비슷한 실력을 가진 학생 40명을 무작위로 4개(A, B, C, D)그룹으로 나누었고, 각 그룹은 다른 교육 방법을 적용했다. 학생들의 학점 결과에는 교육 방법에 따른 차이가 있는지 유의수준 0.5하에서 검정하시오.
- 귀무가설(H0): 네 가지 교육 방법에 의한 학생들의 학점 평균은 동일하다.
- 대립가설(H1): 적어도 두 그룹의 학점 평균은 다르다.

## 1. 기초

In [1]:
# 일원분산은 요인이 1개. 집단은 3개 이상..

import pandas as pd
df = pd.DataFrame({
    'A': [3.5, 4.3, 3.8, 3.6, 4.1, 3.2, 3.9, 4.4, 3.5, 3.3],
    'B': [3.9, 4.4, 4.1, 4.2, 4.5, 3.8, 4.2, 3.9, 4.4, 4.3],
    'C': [3.2, 3.7, 3.6, 3.9, 4.3, 4.1, 3.8, 3.5, 4.4, 4.0],
    'D': [3.8, 3.4, 3.1, 3.5, 3.6, 3.9, 3.2, 3.7, 3.3, 3.4]
})
print(df.head(2))

     A    B    C    D
0  3.5  3.9  3.2  3.8
1  4.3  4.4  3.7  3.4


### 일원 분산 분석

In [2]:
# 일원 분산 분석
from scipy import stats
stats.f_oneway(df['A'], df['B'], df['C'], df['D'])
# 여러 개의 집단을 모두 f_oneway 함수 안에 집어넣고 실행.
# 검정통계량 7.29 / pvalue 0.0006... 귀무가설 기각. 서로 다름.

F_onewayResult(statistic=7.2969837587007, pvalue=0.0006053225519892207)

In [5]:
# 정규성, 등분산, 일원 분산 분석
# 단, 정규성과 등분산을 만족해야 하므로 이 전제 조건이 없는 경우에는 샤피로 윌크와 레빈을 사용하여 정규성과 등분산 검정이 필요함.

# Shapiro-Wilk(샤피로-윌크) 정규성 검정
print(stats.shapiro(df['A']))
print(stats.shapiro(df['B']))
print(stats.shapiro(df['C']))
print(stats.shapiro(df['D']))
# 3가지 모두 pvalue가 0.05를 넘으므로 정규성을 만족함.

# Levene(레빈) 등분산 검정
print(stats.levene(df['A'], df['B'], df['C'], df['D']))
# 4가지 모두 레빈 등분산 검정을 실행한 결과 p-value는 0.2이므로 0.05초과.
# 즉 등분산을 만족한다고 할 수 있음.

# 일원 분산 분석
stats.f_oneway(df['A'],df['B'], df['C'],df['D'])
# 평균에는 차이가 있다.



ShapiroResult(statistic=0.949882447719574, pvalue=0.667110025882721)
ShapiroResult(statistic=0.934644877910614, pvalue=0.49509894847869873)
ShapiroResult(statistic=0.9871343374252319, pvalue=0.9919547438621521)
ShapiroResult(statistic=0.9752339720726013, pvalue=0.9346861243247986)
LeveneResult(statistic=1.5433829973707245, pvalue=0.22000894224209636)


F_onewayResult(statistic=7.2969837587007, pvalue=0.0006053225519892207)

## 2. 심화

In [7]:
# 데이터 재구조화 (긴 형태)
# A / B / C / D였던 구조를 세로로 변경함. -> 집단과 값으로 재구조할 수 있도록 melt라는 함수를 사용.
import pandas as pd
df_melt = df.melt()
df_melt.head()

,variable,value
0,A,3.5
1,A,4.3
2,A,3.8
3,A,3.6
4,A,4.1


### 분산분석 테이블

In [12]:
# ANOVA 테이블
from statsmodels.formula.api import ols

from statsmodels.stats.anova import anova_lm #anova 선형회귀 모델
model = ols('value ~ variable', data = df_melt).fit() #순서는 독립변수 우선, 그 후 종속변수를 적어줌. ()
anova_lm(model)

,df,sum_sq,mean_sq,F,PR(>F)
variable,3.0,2.35875,0.78625,7.296984,0.000605
Residual,36.0,3.87900,0.10775,NaN,NaN


### 사후검정
- 목적: 어떤 그룹들 간에 통계적으로 유의미한 차이가 있는지 구체적으로 파악하는 것

In [15]:
# Tukey HSD (투키)
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
tukey_result = pairwise_tukeyhsd(df_melt['value'], df_melt['variable'], alpha = 0.05)
print(tukey_result)
# 0.05 이상이라면 유의미한 차이가 없음. 0.05 미만이라면 유의미한 차이가 존재 (귀무기각0)
# Bonferroni(본페로니)



Multiple Comparison of Means - Tukey HSD, FWER=0.05 
group1 group2 meandiff p-adj   lower   upper  reject
----------------------------------------------------
     A      B     0.41 0.0397  0.0146  0.8054   True
     A      C     0.09 0.9273 -0.3054  0.4854  False
     A      D    -0.27 0.2722 -0.6654  0.1254  False
     B      C    -0.32 0.1483 -0.7154  0.0754  False
     B      D    -0.68 0.0003 -1.0754 -0.2846   True
     C      D    -0.36 0.0852 -0.7554  0.0354  False
----------------------------------------------------


### 크루스칼-왈리스 검정 (비모수 검정)

In [17]:
import pandas as pd
from scipy import stats

# 데이터
df = pd.DataFrame({
    'A': [10.5, 11.3, 10.8, 10.6, 11.1, 10.2, 10.9, 11.4, 10.5, 10.3],
    'B': [10.9, 11.4, 11.1, 11.2, 11.5, 10.8, 11.2, 10.9, 11.4, 11.3],
    'C': [10.2, 10.7, 10.6, 10.9, 11.3, 11.1, 10.8, 10.5, 11.4, 11.0],
    'D': [13.8, 10.4, 10.1, 10.5, 10.6, 10.9, 10.2, 10.7, 10.3, 10.4]
})

# 데이터 한 개를 변경하여 정규성을 만족하지 않도록 변경.

# 정규성 검정
print(stats.shapiro(df['A']))
print(stats.shapiro(df['B']))
print(stats.shapiro(df['C']))
print(stats.shapiro(df['D']))
#D의 경우 p-value가 -05로 매우 작은 수. (소숫점 5칸 아래로 이동)
#D는 정규성을 만족하지 않으므로 Kruskal-Wallis 검정을 수행한다.
# Kruskal-Wallis 검정
stats.kruskal(df['A'], df['B'], df['C'], df['D'])

ShapiroResult(statistic=0.949882447719574, pvalue=0.667110025882721)
ShapiroResult(statistic=0.934644877910614, pvalue=0.49509894847869873)
ShapiroResult(statistic=0.9871343374252319, pvalue=0.9919547438621521)
ShapiroResult(statistic=0.5759974718093872, pvalue=2.8656615540967323e-05)


KruskalResult(statistic=11.183607021517561, pvalue=0.010773365310213669)

# 이원 분산 분석

가정에서 재배하고 있는 네 가지 토마토 종자(A, B, C, D)에 대해 세 가지 종류의 비료 (11, 12, 13)를 사용하여 재배된 토마토 수를 조사하였다. 종자 및 비료 종류 간의 토마토 수의 차이가 있는지 유의수준 0.05하에서 검정하시오.
(단, 정규성, 등분산성에 만족한 데이터)
- 종자 (주 효과)
    - 귀무가설(H0): 종자 간의 토마토 수에 차이가 없다.
    - 대립가설(H1): 적어도 하나의 종자에서 토마토 수에 차이가 있다.
- 비료 (주 효과)
    - 귀무가설(H0): 비료 종류 간의 토마토 수에 차이가 없다.
    - 대립가설(H1): 적어도 하나의 비료 종류에서 토마토 수에 차이가 있다.
- 상호작용 효과:
    - 귀무가설(H0): 종자와 비료 간의 상호작용은 토마토 수에 영향을 미치지 않는다.
    - 대립가설(H1): 종자와 비료 간의 상호작용은 토마토 수에 영향을 미친다.


## 1. 기초

In [ ]:
import pandas as pd
df = pd.read_csv("tomato.csv")
print(df.head())
print(df.shape)

### 이원 분산 분석

In [ ]:
# anova 테이블


In [ ]:
# 범주형 데이터 처리


In [ ]:
# 일반표기법 format(지수표기법, '.10f')


In [ ]:
# formula * 활용


## 2. 심화

### 사후검정

In [ ]:
# 이원 분산 분석 수행


In [ ]:

# Tukey HSD


In [ ]:
# Bonferroni


### [참고] 정규성, 등분산

In [ ]:
from scipy.stats import shapiro

cond_tree_A = df['종자'] == 'A'
cond_tree_B = df['종자'] == 'B'
cond_tree_C = df['종자'] == 'C'
cond_tree_D = df['종자'] == 'D'

cond_fert_1 = df['비료'] == 11
cond_fert_2 = df['비료'] == 12
cond_fert_3 = df['비료'] == 13

print(shapiro(df[cond_tree_A & cond_fert_1]['토마토수']))
print(shapiro(df[cond_tree_A & cond_fert_2]['토마토수']))
print(shapiro(df[cond_tree_A & cond_fert_3]['토마토수']))

print(shapiro(df[cond_tree_B & cond_fert_1]['토마토수']))
print(shapiro(df[cond_tree_B & cond_fert_2]['토마토수']))
print(shapiro(df[cond_tree_B & cond_fert_3]['토마토수']))

print(shapiro(df[cond_tree_C & cond_fert_1]['토마토수']))
print(shapiro(df[cond_tree_C & cond_fert_2]['토마토수']))
print(shapiro(df[cond_tree_C & cond_fert_3]['토마토수']))

print(shapiro(df[cond_tree_D & cond_fert_1]['토마토수']))
print(shapiro(df[cond_tree_D & cond_fert_2]['토마토수']))
print(shapiro(df[cond_tree_D & cond_fert_3]['토마토수']))

In [ ]:
from scipy.stats import levene
print(levene(df[cond_tree_A]['토마토수'],
             df[cond_tree_B]['토마토수'],
             df[cond_tree_C]['토마토수'],
             df[cond_tree_D]['토마토수']))
print(levene(df[cond_fert_1]['토마토수'],
             df[cond_fert_2]['토마토수'],
             df[cond_fert_3]['토마토수']))